In [ ]:
import pandas as pd

csv_path = r'C:\Users\kosta\Desktop\EPL448\archive\accepted_2007_to_2018q4.csv\accepted_2007_to_2018Q4.csv'
pd.set_option('display.max_columns', None)

# Load a small sample first to inspect schema without exhausting memory.
df = pd.read_csv(csv_path, nrows=5, low_memory=True)
df.head()


In [23]:
print(df.dtypes)

id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object


In [25]:
df.shape

(10000, 151)

In [28]:
print(df.info(verbose =True, max_cols = True,show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 151 columns):
 #    Column                                      Non-Null Count  Dtype  
---   ------                                      --------------  -----  
 0    id                                          10000 non-null  int64  
 1    member_id                                   0 non-null      float64
 2    loan_amnt                                   10000 non-null  float64
 3    funded_amnt                                 10000 non-null  float64
 4    funded_amnt_inv                             10000 non-null  float64
 5    term                                        10000 non-null  object 
 6    int_rate                                    10000 non-null  float64
 7    installment                                 10000 non-null  float64
 8    grade                                       10000 non-null  object 
 9    sub_grade                                   10000 non-null  object 
 10

## Exploratory Data Analysis (EDA)
This section builds a memory-safe EDA workflow for the LendingClub dataset.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 200)


In [ ]:
# Memory-safe EDA sample using chunked reading.
target_rows = 80_000   # lower this (e.g., 20_000) if memory is still tight
chunk_size = 40_000

parts = []
rows_collected = 0

for chunk in pd.read_csv(csv_path, chunksize=chunk_size, low_memory=True):
    remaining = target_rows - rows_collected
    if remaining <= 0:
        break

    if len(chunk) <= remaining:
        take = chunk
    else:
        take = chunk.sample(n=remaining, random_state=42)

    parts.append(take)
    rows_collected += len(take)

eda_df = pd.concat(parts, ignore_index=True)

print(f'EDA sample shape: {eda_df.shape}')
print(f'Approx sample memory: {eda_df.memory_usage(deep=True).sum() / (1024**2):.2f} MB')
eda_df.head()


### Clean Text Columns Into Numeric
Convert binary and numeric-like text columns into numeric features for analysis/modeling.


In [ ]:
# Create a cleaned copy so raw eda_df stays unchanged
eda_num = eda_df.copy()
conversion_log = []

def maybe_numeric(series, min_success=0.85):
    s = series.astype(str).str.strip()
    s = s.replace({'': np.nan, 'nan': np.nan, 'None': np.nan})

    # Remove common non-numeric text/symbols seen in loan datasets
    s = (s
         .str.replace('%', '', regex=False)
         .str.replace('$', '', regex=False)
         .str.replace(',', '', regex=False)
         .str.replace('months', '', regex=False)
         .str.replace('month', '', regex=False)
         .str.replace('years', '', regex=False)
         .str.replace('year', '', regex=False)
         .str.replace('<', '', regex=False)
         .str.replace('+', '', regex=False)
         .str.strip())

    num = pd.to_numeric(s, errors='coerce')
    valid = series.notna().sum()
    success = num.notna().sum() / valid if valid else 0
    return num, success >= min_success, success

for col in eda_num.select_dtypes(include='object').columns:
    raw = eda_num[col]
    lowered = raw.astype(str).str.strip().str.lower()
    unique_vals = set(v for v in lowered.dropna().unique() if v != 'nan')

    # 1) Explicit binary text mapping
    binary_map = {'yes': 1, 'no': 0, 'y': 1, 'n': 0, 'true': 1, 'false': 0, 't': 1, 'f': 0}
    if unique_vals and unique_vals.issubset(set(binary_map.keys())):
        eda_num[col] = lowered.map(binary_map).astype('Int64')
        conversion_log.append((col, 'binary_map', len(unique_vals), 1.0))
        continue

    # 2) Any 2-category text column -> 0/1 automatically
    if 1 < len(unique_vals) <= 2:
        cats = sorted(unique_vals)
        auto_map = {cats[0]: 0, cats[1]: 1}
        eda_num[col] = lowered.map(auto_map).astype('Int64')
        conversion_log.append((col, f'auto_binary:{auto_map}', len(unique_vals), 1.0))
        continue

    # 3) Numeric-like text conversion
    parsed, ok, success = maybe_numeric(raw, min_success=0.85)
    if ok:
        eda_num[col] = parsed
        conversion_log.append((col, 'text_to_numeric', len(unique_vals), round(success, 3)))

conv_df = pd.DataFrame(conversion_log, columns=['column', 'rule', 'unique_values', 'parse_success'])
print(f'Converted columns: {len(conv_df)}')
display(conv_df.sort_values(['rule', 'column']).head(60))

# Use eda_num from now on for numeric-heavy analysis
numeric_cols = eda_num.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = eda_num.select_dtypes(exclude=[np.number]).columns.tolist()
print('Numeric columns after conversion:', len(numeric_cols))
print('Remaining non-numeric columns:', len(cat_cols))

# Keep downstream cells compatible
eda_df = eda_num


In [ ]:
# Basic structure
print('Rows, Columns:', eda_df.shape)
print('Duplicate rows:', eda_df.duplicated().sum())

dtype_counts = eda_df.dtypes.astype(str).value_counts()
display(dtype_counts.to_frame('count'))

eda_df.describe(include='all').transpose().head(30)


In [ ]:
# Missing values
missing_pct = (eda_df.isna().mean() * 100).sort_values(ascending=False)
missing_top = missing_pct[missing_pct > 0].head(25)
display(missing_top.to_frame('missing_%'))

if not missing_top.empty:
    plt.figure(figsize=(10, 8))
    sns.barplot(x=missing_top.values, y=missing_top.index, hue=missing_top.index, legend=False, palette='viridis')
    plt.title('Top Missing Columns (%)')
    plt.xlabel('Missing %')
    plt.ylabel('Column')
    plt.tight_layout()
    plt.show()


In [ ]:
# Separate numeric and categorical features
numeric_cols = eda_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = eda_df.select_dtypes(exclude=[np.number]).columns.tolist()
print('Numeric columns:', len(numeric_cols))
print('Categorical columns:', len(cat_cols))

display(eda_df[numeric_cols].describe().transpose().head(20))
if cat_cols:
    display(eda_df[cat_cols].describe().transpose().head(20))


In [ ]:
# Target and key column distributions (if present)
candidate_targets = ['loan_status', 'grade', 'sub_grade', 'term', 'home_ownership', 'purpose']
for col in candidate_targets:
    if col in eda_df.columns:
        print(f'\n{col} value counts (top 15):')
        display(eda_df[col].value_counts(dropna=False).head(15).to_frame('count'))


In [ ]:
# Numeric distributions for important columns if available
priority_numeric = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high', 'revol_util']
plot_cols = [c for c in priority_numeric if c in eda_df.columns]

if not plot_cols:
    # fallback: use first 6 numeric columns
    plot_cols = numeric_cols[:6]

n = len(plot_cols)
if n > 0:
    rows = (n + 1) // 2
    fig, axes = plt.subplots(rows, 2, figsize=(14, 4 * rows))
    axes = np.array(axes).reshape(-1)

    for i, col in enumerate(plot_cols):
        s = eda_df[col].dropna()
        sns.histplot(s, kde=True, ax=axes[i], bins=40, color='steelblue')
        axes[i].set_title(f'Distribution: {col}')

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Correlation heatmap on top-variance numeric columns
if len(numeric_cols) > 1:
    var_rank = eda_df[numeric_cols].var(numeric_only=True).sort_values(ascending=False)
    corr_cols = var_rank.head(15).index.tolist()
    corr = eda_df[corr_cols].corr(numeric_only=True)

    plt.figure(figsize=(12, 9))
    sns.heatmap(corr, cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap (Top Variance Numeric Columns)')
    plt.tight_layout()
    plt.show()


In [ ]:
# Chunk-based full-file pass for scalable missing-value audit
all_cols = pd.read_csv(csv_path, nrows=0).columns.tolist()
chunk_size = 100_000
row_count = 0
missing_counts = pd.Series(0, index=all_cols, dtype='int64')

for chunk in pd.read_csv(csv_path, chunksize=chunk_size, low_memory=True):
    row_count += len(chunk)
    missing_counts = missing_counts.add(chunk.isna().sum(), fill_value=0)

full_missing_pct = (missing_counts / row_count * 100).sort_values(ascending=False)
print(f'Full dataset row count (chunk scan): {row_count:,}')
display(full_missing_pct.head(25).to_frame('missing_%'))
